# Make BIDS root folder structure for a new EEG study 
(or new BIDS structure for an old study)

Creates the following empty folder structure for a new study:

```
— code
    - import
    - preproc
    - analysis

- derivatives
    - preproc_pipeline_1
        - s001
        - s002
    - preproc_pipeline_2
    - statistical_analysis_1

- rawdata
    - s001
    - s002
    
- README.md
    
- sourcedata
    - eeg
    - behav

```

- `code` contains all code used to operate on the data sets, including file conversion/import, preprocessing, and analysis
- `sourcedata` contains the original, unprocessed data files, organized into subfolders by modality (e.g., EEG, behavioural log files). These files should be write-protected as the original data should never be modified or deleted.
- `rawdata` contains BIDS-compliant raw datasets. These are typically the output of an import script that operates on data in `sourcedata` and converts it to BIDS-compliant formats.
- `derivatives` contain versions of the data that are processed by scripts in the `code` folder. Separate folders in `derivatives` should be made for different types of outputs, e.g., the output of a preprocessing pipeline would go in one folder, but a statistical analysis on that preprocessed data in another. This organization makes it easy to perform different batches of operations on data and compare the results, as well as keeping things organized. Within a `derivatives` subfolder, data should be organized by subject, potentially under subfolders for the outputs of different preprocessing steps, if these are saved.
- `README.md` is a Markdown file which you should edit to include a descirption of the study and experimental paradigm. This provides the context in which the data can be understood.  


In [ ]:
import os
import os.path as op
import json
import yaml
from yaml import CLoader as Loader
from mne_bids import make_dataset_description, update_sidecar_json, BIDSPath

## BIDS Project Parameters
Used to store metadata and for file/folder names

Read from `config.yml`

In [ ]:
# this shouldn't change if you run this script from its default location in code/import
bids_root = op.abspath('.')

cfg_file = op.join(bids_root, 'config.yml')
with open(cfg_file, 'r') as f:
    config = yaml.load(f, Loader=Loader)

study_name = config['study_name']
reb_info = config['reb_info']
authors = config['authors']
instn = config['instn']
instn_addr = config['instn_addr']
instn_dept = config['instn_dept']
funding = config['funding'] 
license = config['license']
data_type = config['data_type']
line_freq = config['line_freq']

## Create study-level metadata

In [ ]:
make_dataset_description(path=bids_root,
                         name=study_name,
                         data_license=license,
                         authors=authors,
                         funding=funding,
                        )

# --- Update metadata with additional info --- 
entries = {'EthicsApprovals':reb_info,
           'InstitutionName':instn,
           'InstitutionAddress':instn_addr,
           'InstitutionalDepartmentName':instn_dept,
           'PowerLineFrequency':line_freq,
          }   
sidecar_path = op.join(bids_root, 'dataset_description.json')
with open(sidecar_path, 'r') as f:
    info = json.load(f)        
info = {**info, **entries}
with open(sidecar_path, 'w') as f:
    info = json.dump(info, f)